In [52]:
# install.packages("magrittr")
# install.packages("purrr")
# install.packages("glue")
# install.packages("stringr")

# install.packages("devtools")
# devtools::install_github("soodoku/tuber", build_vignettes = FALSE)
library(tuber)
library(tidyverse)
library(magrittr) # better handling of pipes
library(purrr) # to work with lists and map functions
library(glue) # to paste strings
library(stringr) # to hand strings
library(rvest) # rvest makes scraping easier
library(polite) # polite is the "polite" version of rvest
library(xml2) # makes it easier to work with HTML and XML from R

Warning message:
"package 'devtools' is in use and will not be installed"



cli      (3.3.0 -> 3.4.1 ) [CRAN]
vctrs    (0.4.1 -> 0.4.2 ) [CRAN]
tibble   (3.1.7 -> 3.1.8 ) [CRAN]
purrr    (0.3.4 -> 0.3.5 ) [CRAN]
dplyr    (1.0.9 -> 1.0.10) [CRAN]
jsonlite (1.8.0 -> 1.8.2 ) [CRAN]
curl     (4.3.2 -> 4.3.3 ) [CRAN]
tidyr    (1.2.0 -> 1.2.1 ) [CRAN]


Installing 8 packages: cli, vctrs, tibble, purrr, dplyr, jsonlite, curl, tidyr

Warning message:
"packages 'tibble', 'purrr', 'dplyr', 'tidyr' are in use and will not be installed"
Installing packages into 'C:/Users/ezeck/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)



package 'cli' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'cli'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\ezeck\AppData\Local\R\win-library\4.2\00LOCK\cli\libs\x64\cli.dll to C:\Users\ezeck\AppData\Local\R\win-library\4.2\cli\libs\x64\cli.dll: Permission denied"
Warning message:
"restored 'cli'"


...

Warning message:
"cannot remove prior installation of package 'jsonlite'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\ezeck\AppData\Local\R\win-library\4.2\00LOCK\jsonlite\libs\x64\jsonlite.dll to C:\Users\ezeck\AppData\Local\R\win-library\4.2\jsonlite\libs\x64\jsonlite.dll: Permission denied"
Warning message:
"restored 'jsonlite'"


package 'curl' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'curl'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\ezeck\AppData\Local\R\win-library\4.2\00LOCK\curl\libs\x64\curl.dll to C:\Users\ezeck\AppData\Local\R\win-library\4.2\curl\libs\x64\curl.dll: Permission denied"
Warning message:
"restored 'curl'"



The downloaded binary packages are in
	C:\Users\ezeck\AppData\Local\Temp\Rtmpwr4SRO\downloaded_packages
* checking for file 'C:\Users\ezeck\AppData\Local\Temp\Rtmpwr4SRO\remotes222413d434c6\soodoku-tuber-c609b27/DESCRIPTION' ... OK
* preparing 'tuber':
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Omitted 'LazyData' from DESCRIPTION
* building 'tuber_0.9.9.tar.gz'



Installing package into 'C:/Users/ezeck/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)



In [34]:
# The website in html
page = read_html("https://www.audioculture.co.nz/music_index?category=Person")

# Profile names
Profile = page %>% html_nodes(".skippy-col-link") %>% html_text()

# Profile links
Link = page %>% html_nodes(".skippy-col-link") %>% html_attr('href')

# Dataframe of profile and associated link
df = data.frame(Profile, Link)

# Prints NZ music profiles based on chars of given string
create_profile_list = function(cafe_name) {
    # Removes special symbols and whitespace from string
    cleaned_string = gsub("([^A-Za-z0-9])+", "", cafe_name)

    # Splits string into list for iteration
    char_list = strsplit(cleaned_string, "")[[1]]

    # The list of NZ profiles based on cafe name
    profile_list = list()

    for (char in char_list) {
        # Converts char into upper case for REGEX pattern
        upper_char = toupper(char)

        # Converts char into lower case for REGEX pattern
        lower_char = tolower(char)

        # Creates list of profiles that match REGEX pattern (Starts with given char)
        match_list = grep(glue('^[{upper_char}{lower_char}]'), Profile, value = TRUE)

        # If the match list is not empty choose random profile from match list
        if (length(match_list) != 0) {
            profile_list = append(profile_list, sample(match_list, 1))
        }
    }

    return(profile_list)
}

# Takes a profile link as input
# Returns profile's discography with featuring artists as a dataframe
get_discography = function(x) {
    profile_discography_page = glue('https://www.audioculture.co.nz{x}/discography') %>% read_html()

    Album     = profile_discography_page %>% html_nodes("[class='header']") %>% html_text()
    Featuring = profile_discography_page %>% html_nodes("[class='body']")   %>% html_text()
    Year      = profile_discography_page %>% html_nodes("[class='year']")   %>% html_text()

    return(data.frame(Album, Featuring, Year))
}

# Takes a profile name as input
# Returns given name's discography (currenly no error handling if name not in dataframe)
search_profile = function(name) {
    name_link = (df %>% filter(str_detect(Profile, name)))[[2]]

    return(get_discography(name_link))
}

In [ ]:
create_profile_list('Bonobo Café')

[[1]]
[1] "Bird Nest Roys"

[[2]]
[1] "Opensouls"

[[3]]
[1] "Nathan Haines"

[[4]]
[1] "Ormsby Brothers, The"

[[5]]
[1] "Broods"

[[6]]
[1] "Opensouls"

[[7]]
[1] "Crude"

[[8]]
[1] "Alastair Dougal"

[[9]]
[1] "Flak"